# Regression
The goal of this notebook is to familiarize you with different aspects of linear regression. We study:   
- linear regression  
- polynomial regression

# PART 1: Linear Regression
(in class exercises)

In [19]:
import numpy as np
from numpy.random import default_rng

from matplotlib import pyplot as plt
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import clear_output, display, HTML
import ipywidgets as widgets

import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline

# The data
We have N=5 data points (x,y) given as two vectors X and Y.

In [14]:
data_X = np.array([-4,   -2,   1,  2.5,  3.9])
data_Y = np.array([-0.9, -0.4, 1.7, 1.5,  2.05])
print(data_X.shape)

(5,)


In [15]:
plt.plot(data_X, data_Y, '--og', linewidth = 1, markersize=10)
plt.grid()
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Data')

Text(0.5, 1.0, 'Data')

# The model

The goal of the model is to take a value x and predict $\hat{y}$. We use the 'hat' symbol to denote an estimated (predicted) value:

$$
\hat{y} = a \cdot x + b
$$


In Python, we can use the LinearRegression class from the scikit-learn library to express a linear model.  <br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
We will use sciki-learn later in this notebook. First we do fit the data manually.

In [20]:
lin_function = lambda x, a, b : a*x+b

# evaluate the model:
y = lin_function(-4, 0.5, 1.0)
print(y)

#We can pass a vector (array) to the lambda expression 
Y = lin_function(data_X, 0.5, 1.0)
print(Y)

-1.0
[-1.    0.    1.5   2.25  2.95]


In [21]:
# for (interactive) visualization we need to wrap the model and the MSE calculation
# into a more complicated function. 
# You can skip this code.


def plt_linear_model(a = 0.1, b = 0.2, X = data_X, Y = data_Y, show_error_squares=False):
    fig = plt.figure(figsize=(8, 6))
    ax = plt.axes()
    
    y = lin_function(X, a, b)

    if show_error_squares:
        ax.plot(X, y, '-xb', linewidth=2)
    else:
        ax.plot(X, y, '-b', linewidth=2)
        
    ax.plot(X, Y, '--og', linewidth=1)
    ax.axhline(y=0, color='tab:gray')
    ax.axvline(x=0, color='tab:gray')
    ax.grid()
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    N = len(X)

    if show_error_squares:
        Esum = 0.
        for i in range(N):
            xi = X[i]
            yi = Y[i]
            y_hat = lin_function(xi, a, b)
            err = yi-y_hat # difference between data and model estimate
            Esum += (err**2)/2.
            if abs(err) > 0.01 :
                sq = plt.Rectangle( (xi, yi), abs(err), -err , alpha=0.2, color='r')
                ax.add_patch(sq)
                ax.plot([xi, xi], [yi, y_hat], linewidth=3, color='r')
        ax.set_title("Linear Model Params: a={:.3}, b={:.3}\nMean squared error (MSE) = {:.4f}".format(a, b, Esum/N))
    else:
        ax.set_title("Linear Model Params: a={:.3}, b={:.3}".format(a, b))
    return a, b

In [22]:
# Guess some value for slope (parameter a) and intercept (parameter b).
# Compare your guess with the data:
plt_linear_model(a = 0.1, b = 0.5)

(0.1, 0.5)

In [23]:
# plt_linear_model example 2
plt_linear_model(a = 0.2, b = -0.2)

(0.2, -0.2)

## Interactive, manual fitting

Use the sliders to find the line which best approximates the given datapoints.

In [24]:
w = interact(plt_linear_model, 
             a=widgets.FloatSlider(min=-1.0, max=1.0, step=0.02, value=-0.1),
             b=widgets.FloatSlider(min=-2.0, max=2.0, step=0.02, value=+1.5),
             X=fixed(data_X),
             Y=fixed(data_Y),
             show_error_squares = fixed(False))
display(w)

interactive(children=(FloatSlider(value=-0.1, description='a', max=1.0, min=-1.0, step=0.02), FloatSlider(valu…

<function __main__.plt_linear_model(a=0.1, b=0.2, X=array([-4. , -2. ,  1. ,  2.5,  3.9]), Y=array([-0.9 , -0.4 ,  1.7 ,  1.5 ,  2.05]), show_error_squares=False)>

# Evaluating your fit: introducing the Loss Function

How good does your linear model approximate the data? To answer this question, we need a way to measure the "goodness" of the fit. A common quantity to look at, are the squared errors:

For each data point $(x_i, y_i)$ we compare the given value $y_i$ with the value $\hat{y}_i$ predicted by the model. The total error is calculated as the sum (over N given data points) of the squared differences, divided by N. For reason that become clear later, the error is (often) divided by 2.

$$
\begin{align}
\hat{y}_i &= a \cdot x_i + b \\
e_i &= y_i - \hat{y}_i   \\
E &= \frac{1}{2N}\sum_{i=1}^N e_i^2 \\
&= \frac{1}{2N}\sum_{i=1}^N (\hat{y} - (a \cdot x_i + b) )^2
\end{align}
$$

This error is known as the **Mean Squared Error (MSE)** (divided by 2). We can visualize the squared errors $e_i^2$ for different model parameters and gain intuition for the error function:




In [25]:
# example: plot the model and visualize the loss
plt_linear_model(a=0.2, b= 0.5,show_error_squares=True)

(0.2, 0.5)

In [26]:
# example 2: 
plt_linear_model( a=0.5, b=1.0,show_error_squares=True)

(0.5, 1.0)

# Interactive MSE minimization


In [27]:
# Example 3: Using the interactive widget, find the optimal fit

w = interact(plt_linear_model, 
             a=widgets.FloatSlider(min=-0.5, max=0.8, step=0.02, value=0.1),
             b=widgets.FloatSlider(min=-1.0, max=1.0, step=0.02, value=0.5),
             X=fixed(data_X),
             Y=fixed(data_Y),
             show_error_squares = fixed(True))
display(w)

interactive(children=(FloatSlider(value=0.1, description='a', max=0.8, min=-0.5, step=0.02), FloatSlider(value…

<function __main__.plt_linear_model(a=0.1, b=0.2, X=array([-4. , -2. ,  1. ,  2.5,  3.9]), Y=array([-0.9 , -0.4 ,  1.7 ,  1.5 ,  2.05]), show_error_squares=False)>

## Fitting a linear model with scikit-learn
scikit-learn.org is a very useful collection of machine-learning tools. Fitting a linear model to data is very simple.


Before applying sklearn, let's inspect the data format. Is X a matrix?

In [28]:
print(data_X)
print(data_X.shape)
print(data_X.reshape(-1, 1))
print(data_X.reshape(-1, 1).shape)

[-4.  -2.   1.   2.5  3.9]
(5,)
[[-4. ]
 [-2. ]
 [ 1. ]
 [ 2.5]
 [ 3.9]]
(5, 1)


In [29]:
# prepare the data for the fit. Note the reshape function.
data_X_array = data_X.reshape(-1, 1)
display(data_X_array)
display(data_X_array.shape)

array([[-4. ],
       [-2. ],
       [ 1. ],
       [ 2.5],
       [ 3.9]])

(5, 1)

In [ ]:
from sklearn import linear_model

#YOUR CODE HERE:

# create a LinearRegression object:

# fit the model to the data:


In [ ]:
lin_model.coef_

In [ ]:
a = lin_model.coef_[0]
print(a)
b = lin_model.intercept_
print(b)
# display( (a, b) )

In [ ]:
# we can use the model to predict values.
y_hat = lin_model.predict(data_X_array)
display(y_hat)
display(y_hat.shape)

We can now anser the initial question (from the slides: for a new x=0.8, what is y)

In [ ]:
y_hat = lin_model.predict([[0.8]])
print(y_hat)

we can predict multiple y_hat at once:

In [ ]:
y_hat = lin_model.predict(data_X_array)

In [ ]:
# plot the result
plt.plot(data_X, y_hat, ':', label=None, color = 'lightgrey')
plt.plot(data_X_array, y_hat, 'o', label='$\hat{y}$')
plt.plot(data_X_array, data_Y, 'o', label='data')
plt.legend()
plt.xlabel('X (Input, independent Variable)')
plt.ylabel('Y (Output, dependent Variable)')
plt.title('Simple linear regression')

### Exercise:
- Calculate the residuals  
- plot a histogram of residuals


In [ ]:
# your code

# PART 2: Multiple Linear Regression
Download the dataset from Moodle or Kaggle:    
https://www.kaggle.com/code/divan0/multiple-linear-regression/data

In this example we want to explain the price of an appartement using multiple variables (for example size, number of rooms, etc. That is, we predict one response variable (aka target variable, dependent variable etc) from multiple input variables (aka "explanatory variables", feature, independent variables)


Note: Multiple Linear Regression is not the same as Multivariate Regression. We study Multiple Linear Regression

In [ ]:
import pandas
import numpy as np
import seaborn as sb


In [ ]:
house_df = pandas.read_csv('kc_house_data.csv')
print(house_df.info())
house_df.head()

In [ ]:
house_price_data = house_df[['price', 'bedrooms', 'sqft_living', 'yr_built']]
house_price_data = house_price_data.sample(n=5000, random_state=12345, replace=False)
# do some cleaning. Here, we do not care about the optimal values. Just remove some data:
house_price_data = house_price_data[house_price_data.price.notnull() & (house_price_data.price>100)  & (house_price_data.price < 3e6)]
house_price_data = house_price_data[house_price_data.bedrooms.notnull() & (house_price_data.bedrooms >= 1)]
house_price_data = house_price_data[house_price_data.price.notnull() & (house_price_data.sqft_living>10)  & (house_price_data.sqft_living < 5000)]
house_price_data = house_price_data[house_price_data.yr_built.notnull() & (house_price_data.yr_built>1900)  & (house_price_data.yr_built < 2010)]

# head shows the first 5 rows. We use this to verify we correctly loaded the data
house_price_data.head()

Before doing any modelling, it's always a good idea to explore the data:

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.pairplot.html
sb.pairplot(house_price_data)

### Simple Linear Regression
The dependent variable (here price) is explained using only a **single** independent variable: 
  
Model 1:  
$price = a_1\cdot area + intercept_1$  


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
simple_linear_regression_model_area = linear_model.LinearRegression()
simple_linear_regression_model_area.fit(house_price_data[[ 'sqft_living']],house_price_data['price'])
simple_linear_regression_model_area.coef_

predicted_price = simple_linear_regression_model_area.predict(house_price_data[[ 'sqft_living']])
residuals = predicted_price - house_price_data['price']
plt.hist(residuals, bins=100)
plt.title('Histogram of residuals')
plt.xlabel("residual")
plt.ylabel("nr of values in bin")
# MSE_sqft_living = np.sum(residuals**2)/residuals.size
MSE_sqft_living = mean_squared_error(predicted_price, house_price_data['price'])
print('MSE sqft_living model= {}'.format(MSE_sqft_living))



### complex model: linear model with 3 independent variables

In [ ]:
multi_lin_model = linear_model.LinearRegression()

X = house_price_data[[ 'bedrooms', 'sqft_living', 'yr_built']]
Y = house_price_data['price']

multi_lin_model.fit(X,Y)

predicted_price = multi_lin_model.predict(house_price_data[['bedrooms', 'sqft_living', 'yr_built']])
residuals = predicted_price - house_price_data['price']

plt.hist(residuals, bins=100)
plt.title('Histogram of residuals')
plt.xlabel("residual")
plt.ylabel("nr of values in bin")

MSE_multiple = np.sum(residuals**2)/residuals.size
print('MSE multiple linear regression model= {}'.format(MSE_multiple))


model parameters:

In [ ]:
print(multi_lin_model.coef_)
print(multi_lin_model.intercept_)


### Questions and Comments  
- What is the interpretation of a negative weight/coefficient?  
- How do you interprete the intercept ?
- Can we compare the weights? Is 'condition' ~30x more important than 'sqft_living' ?  


In [ ]:
sb.pairplot(house_price_data)

## Exercise (not now. Exercise session)
- Add one more factor to the model.  
- Fit the new model  
- Calculate the predicted values and plot the residuals

In [ ]:
# your code goes here

# PART 3: Polynomial Regression (and feature engineering)

Is this a linear model:  
$y=ax^2 + bx + c$

No. But it is **linear in the parameters**  

We can apply linear regression to arbitrarily complex models, as long as it is linear in the **unknowns**.  
Example, linear in a, b, c:  
$y= a \cdot sin(x) + b \cdot exp( x) $

In [ ]:
# let's generate some random data and recover the parameters:
# create random number generator. We reuse this instance later
rng = default_rng()
fancy_function = lambda X, a, b, s=1 : a*X**3 + b*X**2+ rng.normal(loc=0.0, scale=s, size=X.shape)


In [ ]:
X = np.linspace(-6, 3, num=10)
Y = fancy_function(X, 0.2, 1.2, s=0.5)

In [ ]:
plt.plot(X,Y, '.')
plt.title("Data with a non-linear relationship")
plt.xlabel('x')
plt.ylabel('y')

## Can we still use linear regression to fit a model?
- from observation, we assume a relationship of the type $y = ax^3 + bx^2 +cx + d$  
- X is known. Therefore we can calculate $x^3$, $x^2$, $x^1$ 
- How does this differ from the previous example of multiple linear regression, price = f(surface, year_built, ...) ?


It doesn't

How to proceed:  
1. calculate the "features".
2. apply multiple linear regression to recover the feature weights

In [ ]:
X_features = np.column_stack((X**3, X**2, X))
print(X_features)

In [ ]:
lin_model_polynomial = linear_model.LinearRegression()
lin_model_polynomial.fit(X_features,Y)

In [ ]:
print(lin_model_polynomial.coef_)
print(lin_model_polynomial.intercept_)

## Remarks:
- For Polynomials, check out numpy.polyfit  
  https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html  
- The approach shown here is more general. It also works if we create arbitrary complex features f(X).
- calculating features from data is called **feature engineering**

In [ ]:
np.polyfit(X, Y, 3)

## Exercise feature engineering

Go back to the housing data. Someone made the hypothesis that the prices for large appartements grow superlinearly. Make a simple check for this hypothesis:

Compare the MSE of two simple linear models, each with only one feature:
- model 1: price = w1 x sqft_living + w0  
- model 2: price = w1 x (sqft_living^1.5) + w0. 
